# CAS ADS 2024 - M1 Project

This is the code for the M1 project.

Install the necessary package


In [ ]:
!pip install --upgrade fake-useragent

Import all the necessary libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import time

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# we need fake user agents
from fake_useragent import UserAgent

ua = UserAgent()


ModuleNotFoundError: No module named 'fake_useragent'

Let's make a custom header generation function:

In [ ]:

def get_new_header():
  return {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Referer': 'https://www.homegate.ch/',
    'TE': 'Trailers'
}


# Set up a session with retries and backoff
session = requests.Session()

# Retry strategy for handling potential connection issues
retry_strategy = Retry(
    total=5,  # Number of retries
    backoff_factor=1,  # Time between retries: 1s, 2s, 4s, etc.
    status_forcelist=[429, 500, 502, 503, 504],  # Retry on these HTTP status codes
    allowed_methods=["HEAD", "GET", "OPTIONS"]  # Corrected: method_whitelist -> allowed_methods
)

adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

Let try to generate some fake user agents:


In [ ]:
for i in range(5):
  print(get_new_header())

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate, br', 'DNT': '1', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'Sec-Fetch-Dest': 'document', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'none', 'Sec-Fetch-User': '?1'}
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate, br', 'DNT': '1', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'Sec-Fetch-Dest': 'document', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'none', 'Sec-Fetch-User': '?1'}
{'User-Agent': 'Mozilla/5.0 (Windo

First we need to get all the cantons

In [ ]:
cantons_url = "https://www.homegate.ch/rent/apartment/switzerland"
cantons_results = requests.get(cantons_url, headers=get_new_header())
cantons_results.encoding = 'utf-8'

print(f"Get all the cantons from {cantons_url}")
print(f"Status code: {cantons_results.status_code}")

swiss_request = requests.get("https://www.homegate.ch/rent/apartment/country-switzerland/matching-list")
swiss_request.encoding = 'utf-8'
print(f"Status code of swiss: {swiss_request.status_code}")

Get all the cantons from https://www.homegate.ch/rent/apartment/switzerland
Status code: 200
Status code of swiss: 403


In [ ]:
lk

Get all the cantons:

In [2]:
cantons_page = BeautifulSoup(cantons_results.text)
cantons = []
for arr in cantons_page.select('[class^="row GeoDrillDownLocationsSection_spacer_"]'):
  # get the links to all the cantons
  main_div = arr.findChild()
  link_tag = main_div.findChild()

  # region full name
  canton = link_tag.text

  # we skip regions that are not Swiss canton. It must contain the "Canton" string in its text
  if "Canton" not in canton:
    continue

  # only keep the canton name
  canton = canton.split("Canton")[1].strip()
  link_to_districts = "https://www.homegate.ch" + link_tag.get("href")
  print(f"Link: {link_to_districts}")

  cantons.append({"canton": canton, "url": link_to_districts})

print(f"NB Cantons: {len(cantons)}")
print(f"Cantons: {cantons}")

NameError: name 'BeautifulSoup' is not defined

Get all the districts for each canton:

In [ ]:
for canton in cantons:
  time.sleep(1)
  districts_results = requests.get(canton["url"], headers={'User-Agent': ua.random})
  districts_results.encoding = 'utf-8'

  print(f"Get all the districts from {canton['url']}")
  print(f"Status code: {districts_results.status_code}")

  # some pages return error 403, in this case, we print the error message
  if districts_results.status_code != 200:
    print(f"Error: {districts_results.status_code}")
    continue

  response = BeautifulSoup(districts_results.text)


  print()
#print(canton)

Get all the districts from https://www.homegate.ch/rent/apartment/canton-aargau
Status code: 403
Error: 403
Get all the districts from https://www.homegate.ch/rent/apartment/canton-appenzellausserrhoden
Status code: 200

Get all the districts from https://www.homegate.ch/rent/apartment/canton-appenzellinnerrhoden
Status code: 200

Get all the districts from https://www.homegate.ch/rent/apartment/canton-baselland
Status code: 403
Error: 403
Get all the districts from https://www.homegate.ch/rent/apartment/canton-baselstadt
Status code: 403
Error: 403
Get all the districts from https://www.homegate.ch/rent/apartment/canton-bern
Status code: 403
Error: 403
Get all the districts from https://www.homegate.ch/rent/apartment/canton-fribourg
Status code: 200

Get all the districts from https://www.homegate.ch/rent/apartment/canton-geneva
Status code: 403
Error: 403
Get all the districts from https://www.homegate.ch/rent/apartment/canton-glarus
Status code: 403
Error: 403
Get all the districts 

KeyboardInterrupt: 

Some shared function to clean up

In [ ]:
def remove_special_characters(name):
  name = name.strip() # we get the canton and remove all the whitespaces
  name = name.lower() # we convert the canton to lower case
  name = name.replace(" ", "") # we remove whitespace in the canton name
  name = name.replace("-", "") # we remove dash in the canton name
  name = name.replace(".", "") # we remove dash in the canton name
  name = name.replace("ü", "ue") # we replace the ü with ue
  name = name.replace("ö", "oe") # we replace the ö with oe
  name = name.replace("ä", "ae") # we replace the ä with ae
  name = name.replace("â", "a") # we replace the ä with ae

  return name

Then we parse the result html string into an instance of Beautiful Soup.
All the cantons are stored in a html div tag that has a class="DrillDownButton_content_ozH9V".
So wel'll use this class and extract the list of all cantons.
Beware that the class name may change frequently so we need to take all the DrillDownButton_content_* because there is a random part at the end.
Then we loop.
Also, beware the encoding to UTF-8.
I created a function to clean up the name of the cantons, districts and cities

In [ ]:
cantons_page = BeautifulSoup(cantons_results.text)
cantons = []
for arr in cantons_page.select('[class^="DrillDownButton_content_"]'):
  arr = arr.text # get all the text

  arr = arr.split("Canton") # we split the array by canton because the canton is on the right part
  if len(arr) < 2: # if the array has only 1 element, it means that it's not a swiss canton
    continue;

  cantons.append(remove_special_characters(arr[1])) # we clean up and add the canton to our cantons list

print(f"Cantons: {cantons}")
print(f"We have here {len(cantons)} cantons.")

Cantons: ['aargau', 'appenzellausserrhoden', 'appenzellinnerrhoden', 'baselland', 'baselstadt', 'bern', 'fribourg', 'geneva', 'glarus', 'graubuenden', 'jura', 'lucerne', 'neuchatel', 'nidwalden', 'obwalden', 'schaffhausen', 'schwyz', 'solothurn', 'stgallen', 'thurgau', 'ticino', 'uri', 'valais', 'vaud', 'zug', 'zurich']
We have here 26 cantons.


Ok now that we have the cantons, we can query all the district in the cantons.


In [ ]:
# we create a map with all the cantons with theirs districts

# a district looks like this
# https://www.homegate.ch/buy/apartment/canton-aargau
base_district_url = "https://www.homegate.ch/rent/apartment/canton-"

for canton in cantons:
  districts_url = f"{base_district_url}{canton}"
  print(districts_url)




https://www.homegate.ch/rent/apartment/canton-aargau
https://www.homegate.ch/rent/apartment/canton-appenzellausserrhoden
https://www.homegate.ch/rent/apartment/canton-appenzellinnerrhoden
https://www.homegate.ch/rent/apartment/canton-baselland
https://www.homegate.ch/rent/apartment/canton-baselstadt
https://www.homegate.ch/rent/apartment/canton-bern
https://www.homegate.ch/rent/apartment/canton-fribourg
https://www.homegate.ch/rent/apartment/canton-geneva
https://www.homegate.ch/rent/apartment/canton-glarus
https://www.homegate.ch/rent/apartment/canton-graubuenden
https://www.homegate.ch/rent/apartment/canton-jura
https://www.homegate.ch/rent/apartment/canton-lucerne
https://www.homegate.ch/rent/apartment/canton-neuchatel
https://www.homegate.ch/rent/apartment/canton-nidwalden
https://www.homegate.ch/rent/apartment/canton-obwalden
https://www.homegate.ch/rent/apartment/canton-schaffhausen
https://www.homegate.ch/rent/apartment/canton-schwyz
https://www.homegate.ch/rent/apartment/canton